<a href="https://colab.research.google.com/github/mourodrigo/dataScienceCapstone/blob/master/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Use geopy library to get the latitude and longitude values of Toronto.

<h1 align=center><font size = 5>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

This lab is part of a Peer-graded Assignment of the Applied Data Science Capstone


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [100]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

#### Load and explore the data

The following array has been scrapped from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and pre filtered using google sheets ( http://bit.ly/2nbix1W ) to remove boroughs marked  as "Not assingned".

After that, a [Swift code](https://paiza.io/projects/LchIOVXKowPE84pO5vlWkw?language=swift) has been writed to append the Neighborhood names that belongs to the same Borough.

The result then has been placed to the array below:

In [0]:
array = [["M1B", "Scarborough", "Rouge,Malvern"], ["M1C", "Scarborough", "Highland Creek,Rouge Hill,Port Union"], ["M1E", "Scarborough", "Guildwood,Morningside,West Hill"], ["M1G", "Scarborough", "Woburn"], ["M1H", "Scarborough", "Cedarbrae"], ["M1J", "Scarborough", "Scarborough Village"], ["M1K", "Scarborough", "East Birchmount Park,Ionview,Kennedy Park"], ["M1L", "Scarborough", "Clairlea,Golden Mile,Oakridge"], ["M1M", "Scarborough", "Cliffcrest,Cliffside,Scarborough Village West"], ["M1N", "Scarborough", "Birch Cliff,Cliffside West"], ["M1P", "Scarborough", "Dorset Park,Scarborough Town Centre,Wexford Heights"], ["M1R", "Scarborough", "Maryvale,Wexford"], ["M1S", "Scarborough", "Agincourt"], ["M1T", "Scarborough", "Clarks Corners,Sullivan,Tam O\'Shanter"], ["M1V", "Scarborough", "Agincourt North,L\'Amoreaux East,Milliken,Steeles East"], ["M1W", "Scarborough", "L\'Amoreaux West"], ["M1X", "Scarborough", "Upper Rouge"], ["M2H", "North York", "Hillcrest Village"], ["M2J", "North York", "Fairview,Henry Farm,Oriole"], ["M2K", "North York", "Bayview Village"], ["M2L", "North York", "Silver Hills,York Mills"], ["M2M", "North York", "Newtonbrook,Willowdale"], ["M2N", "North York", "Willowdale South"], ["M2P", "North York", "York Mills West"], ["M2R", "North York", "Willowdale West"], ["M3A", "North York", "Parkwoods"], ["M3B", "North York", "Don Mills North"], ["M3C", "North York", "Flemingdon Park,Don Mills South"], ["M3H", "North York", "Bathurst Manor,Downsview North,Wilson Heights"], ["M3J", "North York", "Northwood Park,York University"], ["M3K", "North York", "CFB Toronto,Downsview East"], ["M3L", "North York", "Downsview West"], ["M3M", "North York", "Downsview Central"], ["M3N", "North York", "Downsview Northwest"], ["M4A", "North York", "Victoria Village"], ["M4B", "East York", "Woodbine Gardens,Parkview Hill"], ["M4C", "East York", "Woodbine Heights"], ["M4E", "East Toronto", "The Beaches"], ["M4G", "East York", "Leaside"], ["M4H", "East York", "Thorncliffe Park"], ["M4J", "East York", "East Toronto"], ["M4K", "East Toronto", "The Danforth West,Riverdale"], ["M4L", "East Toronto", "The Beaches West,India Bazaar"], ["M4M", "East Toronto", "Studio District"], ["M4N", "Central Toronto", "Lawrence Park"], ["M4P", "Central Toronto", "Davisville North"], ["M4R", "Central Toronto", "North Toronto West"], ["M4S", "Central Toronto", "Davisville"], ["M4T", "Central Toronto", "Moore Park,Summerhill East"], ["M4V", "Central Toronto", "Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West"], ["M4W", "Downtown Toronto", "Rosedale"], ["M4X", "Downtown Toronto", "Cabbagetown,St. James Town"], ["M4Y", "Downtown Toronto", "Church and Wellesley"], ["M5A", "Downtown Toronto", "Harbourfront,Regent Park"], ["M5B", "Downtown Toronto", "Ryerson,Garden District"], ["M5C", "Downtown Toronto", "St. James Town"], ["M5E", "Downtown Toronto", "Berczy Park"], ["M5G", "Downtown Toronto", "Central Bay Street"], ["M5H", "Downtown Toronto", "Adelaide,King,Richmond"], ["M5J", "Downtown Toronto", "Harbourfront East,Toronto Islands,Union Station"], ["M5K", "Downtown Toronto", "Design Exchange,Toronto Dominion Centre"], ["M5L", "Downtown Toronto", "Commerce Court,Victoria Hotel"], ["M5M", "North York", "Bedford Park,Lawrence Manor East"], ["M5N", "Central Toronto", "Roselawn"], ["M5P", "Central Toronto", "Forest Hill North,Forest Hill West"], ["M5R", "Central Toronto", "The Annex,North Midtown,Yorkville"], ["M5S", "Downtown Toronto", "Harbord,University of Toronto"], ["M5T", "Downtown Toronto", "Chinatown,Grange Park,Kensington Market"], ["M5V", "Downtown Toronto", "CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara"], ["M5W", "Downtown Toronto", "Stn A PO Boxes 25 The Esplanade"], ["M5X", "Downtown Toronto", "First Canadian Place,Underground city"], ["M6A", "North York", "Lawrence Heights,Lawrence Manor"], ["M6B", "North York", "Glencairn"], ["M6C", "York", "Humewood-Cedarvale"], ["M6E", "York", "Caledonia-Fairbanks"], ["M6G", "Downtown Toronto", "Christie"], ["M6H", "West Toronto", "Dovercourt Village,Dufferin"], ["M6J", "West Toronto", "Little Portugal,Trinity"], ["M6K", "West Toronto", "Brockton,Exhibition Place,Parkdale Village"], ["M6L", "North York", "Downsview,North Park,Upwood Park"], ["M6M", "York", "Del Ray,Keelesdale,Mount Dennis,Silverthorn"], ["M6N", "York", "The Junction North,Runnymede"], ["M6P", "West Toronto", "High Park,The Junction South"], ["M6R", "West Toronto", "Parkdale,Roncesvalles"], ["M6S", "West Toronto", "Runnymede,Swansea"], ["M7A", "Queen\'s Park", "Queen\'s Park"], ["M7R", "Mississauga", "Canada Post Gateway Processing Centre"], ["M7Y", "East Toronto", "Business Reply Mail Processing Centre 969 Eastern"], ["M8V", "Etobicoke", "Humber Bay Shores,Mimico South,New Toronto"], ["M8W", "Etobicoke", "Alderwood,Long Branch"], ["M8X", "Etobicoke", "The Kingsway,Montgomery Road,Old Mill North"], ["M8Y", "Etobicoke", "Humber Bay,King\'s Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea"], ["M8Z", "Etobicoke", "Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor"], ["M9A", "Etobicoke", "Islington Avenue"], ["M9B", "Etobicoke", "Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park"], ["M9C", "Etobicoke", "Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe"], ["M9L", "North York", "Humber Summit"], ["M9M", "North York", "Emery,Humberlea"], ["M9N", "York", "Weston"], ["M9P", "Etobicoke", "Westmount"], ["M9R", "Etobicoke", "Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips"], ["M9V", "Etobicoke", "Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown"], ["M9W", "Etobicoke", "Northwest"]]

Fot the Latitude/Longitude, the following CSV has been downloaded and loaded.

In [102]:
!wget -q -O 'geospatial.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [0]:
with open('geospatial.csv') as csv:
  geospatial = pd.read_csv("geospatial.csv") 


The **Postal Code** now will be used as a Index to the searched on the construction of the dataframe.

In [104]:
geospatial.set_index('Postal Code', inplace=True)
geospatial.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [0]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [106]:
neighborhoods

,PostalCode,Borough,Neighborhood


Then let's loop through the data and fill the dataframe one row at a time.

In [0]:
for data in array:
    neighborhoods = neighborhoods.append({'PostalCode': data[0],
                                          'Borough': data[1],
                                          'Neighborhood': data[2],
                                          'Latitude':geospatial.loc[ data[0] , : ]['Latitude'],
                                          'Longitude':geospatial.loc[ data[0] , : ]['Longitude']}, ignore_index=True)

Quickly examine the resulting dataframe.

In [108]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [109]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [110]:
neighborhoods.shape

(103, 5)

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [111]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [112]:
toronto_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [113]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [114]:
CLIENT_ID = 'XCZZBWCOGQ5BJ5KD3NOGA4J0LW52XU25S4IJCS5Z5GRJOGCV' # your Foursquare ID
CLIENT_SECRET = '5JWA0FLEQE0OEDHAJRXKDD2HPWBSUH5LKUASZXDPP13Q0OQ5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XCZZBWCOGQ5BJ5KD3NOGA4J0LW52XU25S4IJCS5Z5GRJOGCV
CLIENT_SECRET:5JWA0FLEQE0OEDHAJRXKDD2HPWBSUH5LKUASZXDPP13Q0OQ5


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [115]:
toronto_data.loc[0, 'Neighborhood']

'Rouge,Malvern'

Get the neighborhood's latitude and longitude values.

In [116]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [117]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=XCZZBWCOGQ5BJ5KD3NOGA4J0LW52XU25S4IJCS5Z5GRJOGCV&client_secret=5JWA0FLEQE0OEDHAJRXKDD2HPWBSUH5LKUASZXDPP13Q0OQ5&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Send the GET request and examine the resutls

In [118]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d95faf32b274a0039c364f0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [120]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


And how many venues were returned by Foursquare?

In [121]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [123]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


Double-click __here__ for the solution.
<!-- The correct answer is:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [124]:
print(toronto_venues.shape)
toronto_venues.head()

(88, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each neighborhood

In [125]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest,Cliffside,Scarborough Village West",3,3,3,3,3,3
"Dorset Park,Scarborough Town Centre,Wexford Heights",7,7,7,7,7,7
"East Birchmount Park,Ionview,Kennedy Park",7,7,7,7,7,7


#### Let's find out how many unique categories can be curated from all the returned venues

In [126]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 51 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [127]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym Pool,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [128]:
toronto_onehot.shape

(88, 52)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [129]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym Pool,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.25,0.00,0.0,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.25,0.00,0.0,0.000000,0.000000,0.000000
3,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.0,0.000000,0.00,0.125,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.0,0.0,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,0.000000,0.125000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.2,0.100000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000,0.000000,0.1,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.00,0.1,0.000000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.000000,0.000,0.000,0.083333,0.0,0.083333,0.0,0.000000,0.00,0.000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.083333,0.000000,0.000000,0.166667,0.166667,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,0.000000,0.083333,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.00,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.0,0.0,0.000,0.000000,0.285714,0.000000,0.000000,0.000000,0.142857,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,

#### Let's confirm the new size

In [130]:
toronto_grouped.shape

(16, 52)

#### Let's print each neighborhood along with the top 5 most common venues

In [131]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0               Lounge  0.25
1       Breakfast Spot  0.25
2         Skating Rink  0.25
3       Sandwich Place  0.25
4  American Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0           Playground  0.33
1          Coffee Shop  0.33
2                 Park  0.33
3  American Restaurant  0.00
4            Pet Store  0.00


----Birch Cliff,Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0              Lounge  0.12
1              Bakery  0.12
2                Bank  0.12
3     Thai Restaurant  0.12
4  Athletics & Sports  0.12


----Clairlea,Golden Mile,Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2  Intersection   0.1
3  Soccer Field   0.1
4   Bus S

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [133]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Coffee Shop,Playground,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Gym Pool,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Lounge,Caribbean Restaurant,Fried Chicken Joint,Hakka Restaurant,Discount Store,Gym Pool
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Park,Soccer Field,Intersection,Bus Station,Fast Food Restaurant,Metro Station,Convenience Store,General Entertainment


<a id='item4'></a>